# Redesigning the way we run Forte computations

What are common models for running computations?

### Gaussian
A simple optimization in Gaussian
```
# test rhf/STO-3G opt !<- computational level specified here

CO sto-3g

0 1 
C
O 1 R

R 0.955
```

### Psi3
Psi3 used input parsing technology
```
psi: (
  label        = "CH2 RHF-CCSD(T)"
  jobtype      = sp
  wfn          = psimrcc
  reference    = rhf 
  docc         = (2 0 1 1)
  convergence  =   14
  basis        = cc-pVDZ
  zmat =
  (
    C
    H 1 1.12
    H 1 1.12 2 152.7
  )
)

mrcc:(
  corr_wfn      = ccsd_t
  corr_docc     = (2 0 1 1)
)
```

### Psi4 (psithon)
A simple optimization in psi4
```
molecule CO {
0 1 
C
O 1 R

R = 0.955
}

optimize('scf/sto-3g')
```

### Psi4 (python API)
Same as previous example, but via python (e.g., `>>> python input.py`)
```python
import psi4

CO = psi4.geometry("""
0 1
C
O 1 R
R = 0.955
""")

psi4.optimize('scf/sto-3g',molecule=CO)
```

### Pyscf (python API)
```python
import pyscf

mol = pyscf.M(
    atom = 'H 0 0 0; F 0 0 1.1',  # in Angstrom
    basis = 'ccpvdz',
    symmetry = True,
)

myhf = mol.HF()
myhf.kernel()

# Orbital energies, Mulliken population etc.
myhf.analyze()
```


### How does Forte currently work?


```python
def run_forte(name, **kwargs):
    # Code to prepare Forte objects: ref_wfn, state_weights_map, mo_space_info, scf_info, fcidump 

    # Run a method
    job_type = options.get_str('JOB_TYPE')

    if job_type == 'NONE':
        psi4.core.set_scalar_variable('CURRENT ENERGY', 0.0)
        # forte.cleanup()
        return ref_wfn

    if 'FCIDUMP' in options.get_str('INT_TYPE'):
        # Make an integral object from a FCIDUMP file
    else:
        # Make an integral object from the psi4 wavefunction object

    # Rotate orbitals before computation (e.g. localization, MP2 natural orbitals, etc.)
    orb_type = options.get_str("ORBITAL_TYPE")
    if orb_type != 'CANONICAL':
        # ...

    # Run a method
    energy = 0.0

    if (options.get_bool("CASSCF_REFERENCE") or job_type == "CASSCF"):
        if options.get_str('INT_TYPE') == 'FCIDUMP':
            raise Exception('Forte: the CASSCF code cannot use integrals read' ' from a FCIDUMP file')
        casscf = forte.make_casscf(state_weights_map, scf_info, options, mo_space_info, ints)
        energy = casscf.compute_energy()

    if (job_type == "MCSCF_TWO_STEP"):
        state_map = forte.to_state_nroots_map(state_weights_map)

        # create an active space solver object and compute the energy
        as_ints = forte.make_active_space_ints(mo_space_info, ints, "ACTIVE", ["RESTRICTED_DOCC"])

        active_space_solver_type = options.get_str('CASSCF_CI_SOLVER')
        active_space_solver = forte.make_active_space_solver(
            active_space_solver_type, state_map, scf_info, mo_space_info, as_ints, options
        )
        casscf = forte.make_mcscf_two_step(state_weights_map, options, ints, active_space_solver)
        energy, _ = casscf.compute_energy()

    if (job_type == 'NEWDRIVER'):
        energy = forte_driver(state_weights_map, scf_info, options, ints, mo_space_info)
    elif (job_type == 'MR-DSRG-PT2'):
        energy = mr_dsrg_pt2(job_type, forte_objects, ints, options)

    end = time.time()

    psi4.core.set_scalar_variable('CURRENT ENERGY', energy)

    psi4.core.print_out(f'\n\n  Time to prepare integrals: {start - start_pre_ints:12.3f} seconds')
    psi4.core.print_out(f'\n  Time to run job          : {end - start:12.3f} seconds')
    psi4.core.print_out(f'\n  Total                    : {end - start_pre_ints:12.3f} seconds\n')

    if 'FCIDUMP' not in options.get_str('INT_TYPE'):
        if options.get_bool('DUMP_ORBITALS'):
            dump_orbitals(ref_wfn)
        return ref_wfn

```

### The danger

![scratch example](graph/img/timelineforte.png)

### The danger

![scratch example](graph/img/danger.jpg)

### The danger

![scratch example](graph/img/multiverse.png)

*But what is happening outside of quantum chemistry?*

### Scratch
Programming for kids

![scratch example](graph/img/scratch.png)

### Houdini
3D graphics

![Houdini example](graph/img/houdini.png)

### Scikit
Machine learning frameworks

![scikit example](graph/img/scikit.png)

### Fast.ai
Another machine learning framework

```python
from fastai.vision.all import *

# get images
path = untar_data(URLs.PETS)/'images'

# label images
def is_cat(x): return x[0].isupper()
dls = ImageDataLoaders.from_name_func(
    path, get_image_files(path), valid_pct=0.2, seed=42,
    label_func=is_cat, item_tfms=Resize(224))

# optimize a convolutional NN
learn = cnn_learner(dls, resnet34, metrics=error_rate)
learn.fine_tune(1)
```